# 动作捕捉并储存

In [ ]:
"""
基于阈值的方法：
你可以定义一些阈值条件，例如当手部关键点的移动速度低于某个阈值时，认为动作已经结束。
这种方法的优点是可以更准确地划分动作，但缺点是可能会受到噪声和其他干扰的影响。
"""

import cv2
import mediapipe as mp
import pyrealsense2 as rs
import pandas as pd
import numpy as np
import os
import time
import warnings
from datetime import datetime

warnings.filterwarnings('ignore')

def get_current_time():
    return datetime.now().strftime('%Y%m%d_%H%M%S')

def is_start_gesture(hand_landmarks):
    # 拇指尖的坐标
    thumb_tip = [hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].x,
                 hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].y,
                 hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].z]

    # 食指尖的坐标
    index_finger_tip = [hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x,
                        hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y,
                        hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].z]

    # 计算拇指尖和食指尖之间的欧氏距离
    distance = np.sqrt(np.sum(np.square(np.subtract(thumb_tip, index_finger_tip))))

    # 如果距离小于一定的阈值，认为是"开始"手势
    return distance < 0.015

def is_end_gesture(hand_landmarks):
    # 拇指尖的坐标
    thumb_tip = [hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].x,
                 hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].y,
                 hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].z]

    # 小指尖的坐标
    pinky_finger_tip = [hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP].x,
                        hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP].y,
                        hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP].z]

    # 计算拇指尖和小指尖之间的欧氏距离
    distance = np.sqrt(np.sum(np.square(np.subtract(thumb_tip, pinky_finger_tip))))

    # 如果距离小于一定的阈值，认为是"结束"手势
    return distance < 0.015

# 初始化 MediaPipe 和 RealSense
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
pipeline.start(config)

# 创建文件夹
if not os.path.exists('vid'):
    os.makedirs('vid')
if not os.path.exists('excel'):
    os.makedirs('excel')

# 初始化视频文件和 DataFrame
out = None
df_list = []
recording = False
start_detected = False


# 捕获和处理视频帧
with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
    while True:
        frames = pipeline.wait_for_frames()
        color_frame = frames.get_color_frame()
        if not color_frame:
            continue
        image = np.asanyarray(color_frame.get_data())

        # 将图像转为RGB色彩空间，进行姿势估计
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image)

        # 将图像转回BGR色彩空间，进行绘制
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                # 判断是否是开始手势
                if is_start_gesture(hand_landmarks):
                    if not recording and not start_detected:
                        print('Start gesture detected.')
                        out = cv2.VideoWriter('vid/{}.avi'.format(get_current_time()), cv2.VideoWriter_fourcc(*'XVID'), 20.0, (640, 480))
                        df_list = []
                        recording = True
                        start_detected = True

                # 判断是否是结束手势
                elif is_end_gesture(hand_landmarks):
                    if recording and start_detected:
                        print('End gesture detected.')
                        out.release()
                        df = pd.DataFrame(df_list)
                        df.to_csv('excel/{}.csv'.format(get_current_time()), index=False)
                        recording = False
                        start_detected = False

                # 提取关键点值并保存
                if recording:
                    hand_data = []
                    for lm in hand_landmarks.landmark:
                        hand_data.append([lm.x, lm.y, lm.z])
                    df_list.append(hand_data)

        # 将帧写入视频文件
        if recording:
            out.write(image)

        # 显示图像
        cv2.imshow('MediaPipe Hands', image)

        # 如果按下ESC键，退出循环
        if cv2.waitKey(5) & 0xFF == 27:
            break

# 如果在退出循环时仍在录制，保存当前的视频和 DataFrame
if recording:
    # 如果录制超过0.5秒，才保存
    if len(df_list) > 15: # 假设帧率为30fps，那么0.5秒会有15帧
        out.release()
        df = pd.DataFrame(df_list)
        df.to_csv('excel/{}.csv'.format(get_current_time()), index=False)

# 释放资源
pipeline.stop()
cv2.destroyAllWindows()


# 分类信息读取

In [ ]:
import os
import subprocess

print("检查 'vid' 文件夹内容：")
print(os.listdir('vid'))

def rename_file(original_name, append_string):
    # 定义视频和csv文件的路径
    video_path = os.path.join('vid', original_name)
    csv_path = os.path.join('excel', original_name.split('.')[0] + ".csv")

    # 修改视频名
    if os.path.exists(video_path):
        video_new_name = original_name.split('.')[0] + append_string + "." + original_name.split('.')[1]
        video_new_path = os.path.join('vid', video_new_name)
        os.rename(video_path, video_new_path)

    # 修改csv文件名
    if os.path.exists(csv_path):
        csv_new_name = original_name.split('.')[0] + append_string + ".csv"
        csv_new_path = os.path.join('excel', csv_new_name)
        os.rename(csv_path, csv_new_path)

def delete_file(file_name):
    video_path = os.path.join('vid', file_name)
    csv_path = os.path.join('excel', file_name.split('.')[0] + ".csv")

    if os.path.exists(video_path):
        os.remove(video_path)
        print(f"视频 '{file_name}' 已删除.")

    if os.path.exists(csv_path):
        os.remove(csv_path)
        print(f"CSV 文件 '{file_name.split('.')[0]}.csv' 已删除.")

def play_video_with_default_player(video_path):
    if os.name == 'nt':  # for Windows
        os.startfile(video_path)
    else:
        opener = "open" if sys.platform == "darwin" else "xdg-open"  # for macOS and Linux
        subprocess.call([opener, video_path])

def main():
    print("开始执行 main 函数...")
    for video_name in os.listdir('vid'):
        if "_edited" not in video_name:  # 更改筛选条件
            video_path = os.path.join('vid', video_name)
            play_video_with_default_player(video_path)

            print(f"视频 {video_name} 预览完毕!")
            option = input("请选择操作：\n1. 重命名文件\n2. 删除文件\n输入选项（1或2）：")

            if option == '1':
                append_string = "_edited"  # 添加特定后缀表示文件已编辑
                rename_file(video_name, append_string)
            elif option == '2':
                delete_file(video_name)
            else:
                print("无效的选项!")
    print("结束 main 函数执行。")

if __name__ == "__main__":
    main()


In [ ]:
import os
def sync_excel_to_vid():
    # 获取vid和excel目录下的所有文件
    vid_files = os.listdir('vid')
    excel_files = os.listdir('excel')
    
    for excel_file in excel_files:
        # 去掉扩展名的文件名
        base_name = os.path.splitext(excel_file)[0]
        
        # 检查是否有相对应的视频文件
        matched_video_files = [v for v in vid_files if base_name in v]
        
        # 如果找到了匹配的视频文件
        if matched_video_files:
            # 确保只有一个匹配的视频文件
            if len(matched_video_files) == 1:
                matched_video_file = matched_video_files[0]
                new_excel_name = os.path.splitext(matched_video_file)[0] + '.csv'
                
                # 重命名excel文件
                os.rename(os.path.join('excel', excel_file), os.path.join('excel', new_excel_name))
            else:
                print(f"在'vid'目录中找到了多个与 '{base_name}' 匹配的文件，无法确定要使用哪一个。")
        # 如果在vid文件夹中找不到匹配的文件
        else:
            os.remove(os.path.join('excel', excel_file))
            print(f"文件 '{excel_file}' 已从 'excel' 文件夹中删除。")
sync_excel_to_vid()

# 识别模型搭建

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout, BatchNormalization

# 设定视频帧的形状
frame_num = 30  # 时间长度
frame_height = 64  # 高度
frame_width = 64  # 宽度
channels = 3  # 通道数

# 设定分类的类别数
num_classes = 5  # 假设你有5个动作需要分类

# 定义模型
model = Sequential()

# 第一层3D卷积
model.add(Conv3D(64, (3, 3, 3), activation='relu', padding='same', 
                 input_shape=(frame_num, frame_height, frame_width, channels)))
model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))

# 第二层3D卷积
model.add(Conv3D(128, (3, 3, 3), activation='relu', padding='same'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

# 第三层3D卷积
model.add(Conv3D(256, (3, 3, 3), activation='relu', padding='same'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

# 第四层3D卷积
model.add(Conv3D(256, (3, 3, 3), activation='relu', padding='same'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

# 展平
model.add(Flatten())

# 全连接层
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))

# 输出层
model.add(Dense(num_classes, activation='softmax'))

# 编译模型
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

# 打印模型概况
model.summary()
